In [18]:
import sys
sys.path.append("../src")
from mistral_client import run_mistral
from ner_post_processing import parse_entities_promptner, get_token_labels

import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
from collections import defaultdict

In [19]:
dataset = load_dataset("DFKI-SLT/cross_ner", "politics")

class_labels = dataset["validation"].features["ner_tags"].feature.names + ["B-frenchpolitician", "I-frenchpolitician", "B-frenchpoliticalparty", "I-frenchpoliticalparty"]
index2label = {i: label for i, label in enumerate(class_labels)}
label2index = {v: k for k, v in index2label.items()}

label2index

{'O': 0,
 'B-academicjournal': 1,
 'I-academicjournal': 2,
 'B-album': 3,
 'I-album': 4,
 'B-algorithm': 5,
 'I-algorithm': 6,
 'B-astronomicalobject': 7,
 'I-astronomicalobject': 8,
 'B-award': 9,
 'I-award': 10,
 'B-band': 11,
 'I-band': 12,
 'B-book': 13,
 'I-book': 14,
 'B-chemicalcompound': 15,
 'I-chemicalcompound': 16,
 'B-chemicalelement': 17,
 'I-chemicalelement': 18,
 'B-conference': 19,
 'I-conference': 20,
 'B-country': 21,
 'I-country': 22,
 'B-discipline': 23,
 'I-discipline': 24,
 'B-election': 25,
 'I-election': 26,
 'B-enzyme': 27,
 'I-enzyme': 28,
 'B-event': 29,
 'I-event': 30,
 'B-field': 31,
 'I-field': 32,
 'B-literarygenre': 33,
 'I-literarygenre': 34,
 'B-location': 35,
 'I-location': 36,
 'B-magazine': 37,
 'I-magazine': 38,
 'B-metrics': 39,
 'I-metrics': 40,
 'B-misc': 41,
 'I-misc': 42,
 'B-musicalartist': 43,
 'I-musicalartist': 44,
 'B-musicalinstrument': 45,
 'I-musicalinstrument': 46,
 'B-musicgenre': 47,
 'I-musicgenre': 48,
 'B-organisation': 49,
 'I-o

In [20]:
fp = "../data/unlabeled/politics_tasklevel.txt"
fp_scored = "../data/scored/politics_tasklevel_french.csv"

In [21]:
sentences = []
with open(fp, "r") as file:

        while True:
            line = file.readline()  # Read a single line
            if not line:  # Break out of the loop when the end of the file is reached
                break
            line = line.strip()
            sentences.append(line)

In [22]:
len(sentences)

501885

In [23]:
import random

sentences_rand = random.sample(sentences, 10000)

In [24]:
len(sentences_rand)

10000

In [25]:
prompt_french = lambda text: f"""
Dfn: An entity is a person (person), organisation (organisation), french politician (politician), french political party (politicalparty), event (event), election (election), 
country (country), location (location) or other political entity (misc). Dates, times, abstract concepts, adjectives, and verbs are not entities.

Example 1: In the 2014 European Parliament election in France , the National Front won the elections with 24.85 % of the vote , a swing of 18.55 % , winning 24 seats , up from 3 previously .

Answer:
1. 2014 European Parliament election | True | as it is an election (election)
2. France | True | as it is a country (country)
3. National Front | True | as it is a political party (frenchpoliticalparty)

Example 2: The FN received 33.9 % of the votes in the 2017 French presidential election , making it the largest Eurosceptic party in France .

Answer:
1. FN | True | as it is a political party (frenchpoliticalparty)
2. 2017 French presidential election | True | as it is an election (election)
3. Eurosceptic party | True | as it is a political party (frenchpoliticalparty)
4. France | True | as it is a country (country)

Example 3: The 2017 French presidential election caused a radical shift in French politics , as the prevailing parties of The Republicans and Socialists failed to make it to the second round of voting , with far-right Marine Le Pen and political newcomer Emmanuel Macron instead facing each other .

Answer:
1. 2017 French presidential election | True | as it is an election (election)
2. French politics | False | as it is an abstract concept, not a specific entity
3. The Republicans | True | as it is a political party (frenchpoliticalparty)
4. Socialists | True | as it is a political party (frenchpoliticalparty)
5. far-right | False | as it is an adjective describing a political orientation, not a specific entity
6. Marine Le Pen | True | as she is a French politician (frenchpolitician)
7. Emmanuel Macron | True | as he is a French politician (frenchpolitician)
8. political newcomer | False | as it is an abstract concept, not a specific entity

Q. Given the paragraph below, identify a list of possible entities and for each entry explain why it either is or is not an entity.

Paragraph: {text}
"""

In [26]:
scored = defaultdict(list)

for idx, text in enumerate(tqdm(sentences_rand)):
    if (idx + 1) % 100 == 0:
        df_scored = pd.DataFrame(scored)
        df_scored.to_csv(fp_scored, index=False)
    
    try:
        tokens = text.split()
        prompt_input = prompt_french(text)
        output = run_mistral(prompt_input, model="open-mistral-7b")
        ner_tags = get_token_labels(text, parse_entities_promptner(output), label2index)

        scored["tokens"].append(text.split(" "))
        scored["prompt"].append(prompt_input)
        scored["output"].append(output)
        scored["ner_tags"].append(ner_tags)
    except Exception as e:
        print(e)
        continue

df_scored = pd.DataFrame(scored)
df_scored.to_csv(fp_scored, index=False)

  6%|▌         | 561/10000 [28:20<7:12:24,  2.75s/it] 

In [ ]:
df_scored

,tokens,prompt,output,ner_tags
0,"[Gianfranco, Fini, ,, a, moderniser, who, saw,...","\nDfn: An entity is a person (person), organis...",1. Gianfranco Fini | True | as he is a politic...,"[57, 58, 0, 0, 0, 0, 0, 57, 58, 0, 57, 58, 0, ..."
1,"[(, Clockwise, ), Obama, (, standing, ), ,, Da...","\nDfn: An entity is a person (person), organis...","1. Obama | True | as he is a person (person), ...","[0, 0, 0, 51, 0, 0, 0, 0, 51, 52, 0, 51, 52, 0..."
2,"[He, interacted, adeptly, with, allies, such, ...","\nDfn: An entity is a person (person), organis...","1. He | False | as it is a pronoun, not a spec...","[0, 0, 0, 0, 0, 0, 0, 51, 52, 0, 51, 52, 52, 5..."
3,"[The, European, Union, was, formally, establis...","\nDfn: An entity is a person (person), organis...",1. European Union | True | as it is an organiz...,"[0, 0, 0, 0, 0, 0, 0, 0, 29, 30, 0, 0, 0, 0, 0..."
4,"[In, February, 2004, ,, he, became, a, candida...","\nDfn: An entity is a person (person), organis...","1. February 2004 | False | as it is a date, no...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 79, 80, 0, 0, 0, 0..."
...,...,...,...,...
944,"[The, group, ,, composed, of, 73, MEPs, ,, was...","\nDfn: An entity is a person (person), organis...",1. The group | True | as it is an organisation...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 35, 0, 41, 4..."
945,"[Le, Pen, 's, statements, about, a, "", boreal,...","\nDfn: An entity is a person (person), organis...",1. Le Pen | True | as he is a French politicia...,"[79, 80, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
946,"[L, 'Œuvre, Française, was, dissolved, on, 23,...","\nDfn: An entity is a person (person), organis...",1. L 'Œuvre Française | True | as it is an org...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
947,"[In, 2005, ,, she, was, named, as, Secretary, ...","\nDfn: An entity is a person (person), organis...","1. 2005 | False | as it is a date, not an enti...","[0, 0, 0, 51, 0, 0, 0, 41, 42, 42, 42, 42, 0, ..."
